In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
from rom_tools import rom_manager
from rom_tools.address import *
from rom_tools import byte_ops
from rom_tools.compress import decompress

In [88]:
from PIL import Image

In [79]:
rom_m = rom_manager.RomManager("../sm_guinea_pig.smc", "../sm_newlevel.smc")

## Palette

In [80]:
# ucp - Upper Crateria Palette
ucp_snes = 0xc2ad7c
ucp_addr = Address(0)
ucp_addr.from_snes(0xc2ad7c)
ucp_addr

0x212d7c

In [81]:
compressed_ucp = rom_m.read_from_clean(ucp_addr, 1000)
ucp = decompress.decompress(compressed_ucp)
len(ucp)

256

In [82]:
# 15 bit BGR, highest bit unused
def get_rgb(color):
    r_mask = 0b11111
    g_mask = r_mask << 5
    b_mask = r_mask << 10
    # Mask out the correct 5-bit values
    r5 = color & r_mask
    g5 = (color & g_mask) >> 5
    b5 = (color & b_mask) >> 10
    # Upscale them to 8-bit
    r8 = r5 << 3
    g8 = g5 << 3
    b8 = b5 << 3
    return (r8, g8, b8)
    

In [83]:
i = int.from_bytes(ucp[8:10], byteorder='little')
get_rgb(i)

(168, 120, 184)

In [91]:
palette_bytes = rom_m.read_compressed_list(ucp_addr, 128, 2)
palette_colors = list(map(lambda x: get_rgb(int.from_bytes(x, byteorder='little')), palette_bytes))

In [93]:
palette = Image.new("RGBA", (16,8), "black")
for x in range(16):
    for y in range(8):
        index = y * 16 + x
        r,g,b = palette_colors[index]
        if x == 0:
            a = 0
        else:
            a = 255
        palette.putpixel((x,y), (r,g,b,a))
palette.save("palette.png")

## Subtile Sheet

In [188]:
uc_subtile_snes = 0xbac629
#uc_subtile_snes = 0xbaf911
#uc_subtile_snes = 0xbbae9e
uc_subtile_addr = Address(0)
uc_subtile_addr.from_snes(uc_subtile_snes)
uc_subtile_addr

0x1d4629

In [189]:
compressed_uc_subtiles = rom_m.read_from_clean(uc_subtile_addr, 16000)
uc_subtiles = decompress.decompress(compressed_uc_subtiles)
len(uc_subtiles) / 32

576.0

In [190]:
subtiles_bytes = rom_m.read_compressed_list(uc_subtile_addr, 576, 32)

In [193]:
def get_pixel_color_index(tile, x, y, bpp=4):
    palette_index = 0
    for i in range(bpp // 2):
        for j in range(2):
            palette_index |= (tile[y * 2 + i * 0x10 + j] >> 7 - x & 1) << i * 2 + j
    return palette_index

In [194]:
subtiles = Image.new("RGBA", (128, 288), "black")
for index, subtile in enumerate(subtiles_bytes):
    big_x = index % 16
    big_y = index // 16
    for x in range(8):
        for y in range(8):
            palette_index = get_pixel_color_index(subtile, x, y)
            palette_color = palette.getpixel((palette_index, 0))
            subtiles.putpixel((big_x * 8 + x, big_y * 8 + y), palette_color)
subtiles.save("subtiles.png")

## Tile Table